# 1. Importing Libraries

In [136]:
# Import libraries

import pandas as pd
import numpy as np
import os

# 2. Define Path

In [137]:
# create a string of the path for the project folder and assign the string to a variable

folder_path = r'F:\CF\6. Advanced Analytics\Brazilian E-Commerce'

# 3. Import Data Set Order_items

In [138]:
# Import order_reviews into Jupyter as df_reviews

df_reviews = pd.read_csv(os.path.join(folder_path, '02 Data', 'Original Data', 'olist_order_reviews_dataset.csv'), index_col = False)

In [139]:
# Display the first 5 rows of the dataframe

df_reviews.head()

review_id                          order_id  \
0  7bc2406110b926393aa56f80a40eba40  73fc7af87114b39712e6da79b0a377eb   
1  80e641a11e56f04c1ad469d5645fdfde  a548910a1c6147796b98fdf73dbeba33   
2  228ce5500dc1d8e020d8d1322874b6f0  f9e4b658b201a9f2ecdecbb34bed034b   
3  e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   
4  f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1   

   review_score review_comment_title  \
0             4                  NaN   
1             5                  NaN   
2             5                  NaN   
3             5                  NaN   
4             5                  NaN   

                              review_comment_message review_creation_date  \
0                                                NaN  2018-01-18 00:00:00   
1                                                NaN  2018-03-10 00:00:00   
2                                                NaN  2018-02-17 00:00:00   
3              Recebi bem antes do prazo estipulado.  2017-04-21 00:00:00   
4  Parabéns lojas lannister adorei comprar pela I...  2018-03-01 00:00:00   

  review_answer_timestamp  
0     2018-01-18 21:46:59  
1     2018-03-11 03:05:13  
2     2018-02-18 14:36:24  
3     2017-04-21 22:02:06  
4     2018-03-02 10:26:53

In [140]:
# Display the number of rows and columns in the dataframe

df_reviews.shape

(99224, 7)

In [141]:
# returns columns and their data types

df_reviews.dtypes

review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object

# 4. Data Wrangling on Data Set

In [142]:
# Convert ID columns to the more efficient string type

id_cols = ['review_id', 'order_id', 'review_comment_title', 'review_comment_message']

for col in id_cols:
    df_reviews[col] = df_reviews[col].astype('string')

In [143]:
# Converting scores to int8 save smemory because Scores are only 1–5 (small range) 

df_reviews = df_reviews.astype ( { "review_score": "int8" } )

In [144]:
# Convert datetime columns

df_reviews["review_creation_date"] = pd.to_datetime(df_reviews["review_creation_date"])

df_reviews["review_answer_timestamp"] = pd.to_datetime(df_reviews["review_answer_timestamp"])

In [145]:
# Confirm changes

df_reviews.dtypes

review_id                  string[python]
order_id                   string[python]
review_score                         int8
review_comment_title       string[python]
review_comment_message     string[python]
review_creation_date       datetime64[ns]
review_answer_timestamp    datetime64[ns]
dtype: object

In [146]:
# Counting the frequency of the values in column 'review_score'

df_reviews['review_score'].value_counts(dropna=False)

review_score
5    57328
4    19142
1    11424
3     8179
2     3151
Name: count, dtype: int64

In [147]:
# Check if review_ids are unique

df_reviews["review_id"].is_unique

False

In [148]:
# Check if order_ids are unique

df_reviews["order_id"].is_unique

False

In this data set, order_ids and review_ids are not unique. In the following, we show that a combination of order_id and review_id is unique and can be considered as the primary key for this data set.

In [149]:
# Step 1: Create a subset with only the two columns 'review_id' and 'order_id'.

df_subset = df_reviews[['review_id', 'order_id']]

In [150]:
# Step 2: Check for duplicate rows in this subset 

num_duplicates_in_subset = df_subset.duplicated().sum()

# .duplicated() checks for rows that are identical across all columns in the DataFrame it's called on.
# .sum() is used because it treats True as 1 and False as 0, giving a quick count.

In [151]:
# Step 3: Print the results

if num_duplicates_in_subset == 0:
    
    print("Result: No duplicate rows found in the subset.")
    print("This confirms that the combination of 'review_id' and 'order_id' is unique.")
else:
    print(f"Result: Found {num_duplicates_in_subset} duplicate rows in the subset.")
    print("This means there are cases where the exact same ('review_id', 'order_id') pair appears more than once.")

Result: No duplicate rows found in the subset.
This confirms that the combination of 'review_id' and 'order_id' is unique.


The result confirms that the combination of review_id and order_id is unique for every single row.

This means the data is structurally sound. Every review (review_id) is properly linked to a unique order (order_id). The table has a valid composite primary key.

In [152]:
# Finding missing values in dataframe

df_reviews.isnull().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

✅ Columns **review\_id, order\_id, review\_score, review\_creation\_date, and review\_answer\_timestamp** have no missing values.

- The column **review\_comment\_title** has about **87,656** missing values.

- The column **review\_comment\_message** has about **58,247** missing values.

🔹 Interpretation:

- The columns **review\_comment\_title** and **review\_comment\_message** are optional text fields.

- Many customers only give a score (**review\_score**) without writing any text or title.

- So, these "missing values" are actually natural and do not mean poor data quality.


In [153]:
# Replace None or empty strings with np.nan

df_reviews['review_comment_title'] = df_reviews['review_comment_title'].replace(['', 'None'], np.nan)
df_reviews['review_comment_message'] = df_reviews['review_comment_message'].replace(['', 'None'], np.nan)

# Ensure column types are string (object) with NaN for missing

df_reviews['review_comment_title'] = df_reviews['review_comment_title'].astype('string')
df_reviews['review_comment_message'] = df_reviews['review_comment_message'].astype('string')


In [154]:
# Looking for full duplicates within dataframe

df_duplicates = df_reviews[df_reviews.duplicated()]

print ( len( df_duplicates) )

0


 len( df_duplicates) = 0 ---> No duplicates

In [155]:
# Check for mixed types in df_reviews

for col in df_reviews.columns.tolist():
    weird = ( df_reviews[[col]].map(type) != df_reviews[[col]].iloc[0].apply(type) ).any(axis=1)
    if len(df_reviews[weird]) > 0:
        print (f'*** Mixed types data in column: {col} *** Number of affected rows: {len(df_reviews[weird])} ***' )
    else:
        print (f'Column {col}:     No mixed type was found!')

Column review_id:     No mixed type was found!
Column order_id:     No mixed type was found!
Column review_score:     No mixed type was found!
*** Mixed types data in column: review_comment_title *** Number of affected rows: 11568 ***
*** Mixed types data in column: review_comment_message *** Number of affected rows: 40977 ***
Column review_creation_date:     No mixed type was found!
Column review_answer_timestamp:     No mixed type was found!


In [156]:
# Subset 1: Rows where review_comment_title is NaN (missing)

subset_title_nan = df_reviews[df_reviews['review_comment_title'].isna()]

# Subset 2: Rows where review_comment_title is string

subset_title_str = df_reviews[df_reviews['review_comment_title'].apply(lambda x: isinstance(x, str))]

# Subset 3: Rows where review_comment_message is NaN (missing)

subset_message_nan = df_reviews[df_reviews['review_comment_message'].isna()]

# Subset 4: Rows where review_comment_message is string

subset_message_str = df_reviews[df_reviews['review_comment_message'].apply(lambda x: isinstance(x, str))]

# Show how many rows in each subset
print("review_comment_title: - NaN:", len(subset_title_nan))
print("                      - string:", len(subset_title_str))
print('\n')
print("review_comment_message: - NaN:", len(subset_message_nan))
print("                        - string:", len(subset_message_str))


review_comment_title: - NaN: 87656
                      - string: 11568


review_comment_message: - NaN: 58247
                        - string: 40977


**Result:**

The "mixed type" problem happens because the columns `review_comment_title` and `review_comment_message` contain both **strings** (the actual review text) and **missing values** (`NaN`).

So, technically the column has two data types:

* `str` → when a customer wrote a title or message
* `NaN` (special missing value type) → when the customer left it empty

This is normal and not a real data quality issue. It just reflects the fact that many customers gave only a score without writing text.

In [157]:
# returns descriptive statistics of the dataframe

df_reviews.describe()

review_score           review_creation_date  \
count  99224.000000                          99224   
mean       4.086421  2018-01-12 20:49:23.948238336   
min        1.000000            2016-10-02 00:00:00   
25%        4.000000            2017-09-23 00:00:00   
50%        5.000000            2018-02-02 00:00:00   
75%        5.000000            2018-05-16 00:00:00   
max        5.000000            2018-08-31 00:00:00   
std        1.347579                            NaN   

             review_answer_timestamp  
count                          99224  
mean   2018-01-16 00:23:56.977938688  
min              2016-10-07 18:32:28  
25%    2017-09-27 01:53:27.249999872  
50%       2018-02-04 22:41:47.500000  
75%       2018-05-20 12:11:21.500000  
max              2018-10-29 12:27:35  
std                              NaN

# 5. Exporting Dataframe

In [1]:
# Exporting df_reviews dataframe as “order_reviews_checked.csv” in the “Prepared Data” folder

df_reviews.to_csv(os.path.join(folder_path, '02 Data', 'Prepared Data', 'reviews_checked.csv'))

NameError: name 'df_reviews' is not defined